<a href="https://colab.research.google.com/github/bhgtankita/MYWORK/blob/master/Copy_of_Assignment_3C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
!ls 'gdrive/My Drive/Weights'

In [0]:
# https://keras.io/
!pip install -q keras
import keras

import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint,CSVLogger
from keras.preprocessing.image import ImageDataGenerator

# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 10
num_filter = 20

Using TensorFlow backend.


In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 21s 0us/step


In [0]:
def space_to_depth_x2(x):
    return tf.space_to_depth(x, block_size=8)

In [0]:
compression = 2
num_filter = 16

In [0]:
def add_conv_block(input, num_filter = 16):
  global compression
  temp = input
  compression = 2
  
  for i in range(5):
    BatchNorm = BatchNormalization()(temp)
    Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(BatchNorm)
    lrelu = LeakyReLU(alpha=0.1)(Conv2D_3_3)
    compression *= 2
    temp = lrelu
  
  temp = MaxPooling2D(pool_size=(2, 2))(temp)

  return temp

In [0]:
input = Input(shape=(img_height, img_width, channel,))

First_Block = add_conv_block(input, num_filter)
Second_Block = add_conv_block(First_Block, num_filter)
Third_Block = add_conv_block(Second_Block, num_filter)
Fourth_Block = add_conv_block(Third_Block, num_filter)

#First_Block_Reduce = space_to_depth_x2(First_Block)

First_Block_Reduce = MaxPooling2D(pool_size=(2, 2))(First_Block)
First_Block_Reduce = MaxPooling2D(pool_size=(2, 2))(First_Block_Reduce)
First_Block_Reduce = MaxPooling2D(pool_size=(2, 2))(First_Block_Reduce)

concat = Concatenate(axis=-1)([First_Block_Reduce,Fourth_Block])

#lastlayer = Flatten()(concat)
#output = Dense(num_classes, activation='softmax')(lastlayer)

layer1 = BatchNormalization(name='norm_1')(concat)
layer1 = Conv2D(512, (1,1), strides=(1,1), padding='same', name='conv_1', use_bias=False)(layer1)
layer1 = LeakyReLU(alpha=0.1)(layer1)

layer2 = BatchNormalization(name='norm_2')(layer1)
layer2 = Conv2D(64, (1,1), strides=(1,1), padding='same', name='conv_2', use_bias=False)(layer2)
layer2 = LeakyReLU(alpha=0.1)(layer2)

layer3 = BatchNormalization(name='norm_3')(layer2)
layer3 = Conv2D(32, (1,1), strides=(1,1), padding='same', name='conv_3', use_bias=False)(layer3)
layer3 = LeakyReLU(alpha=0.1)(layer3)

layer4 = BatchNormalization(name='norm_4')(layer3)
layer4 = Conv2D(10, (1,1), strides=(1,1), padding='same', name='conv_4', use_bias=False)(layer4)
layer4 = LeakyReLU(alpha=0.1)(layer4)

output = AveragePooling2D(pool_size=(2,2))(layer4)
output = Activation('softmax')(output)
output = Flatten()(output)



Instructions for updating:
Colocations handled automatically by placer.


In [0]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 3)    12          input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 32)   864         batch_normalization_1[0][0]      
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 32, 32, 32)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
batch_norm

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

In [0]:
gen = ImageDataGenerator( #featurewise_center=True,
                          #featurewise_std_normalization=True,
                          rotation_range=20, 
                          width_shift_range=0.2, 
                          #shear_range=0.2,
                          height_shift_range=0.2, 
                          zoom_range=0.15,
                          horizontal_flip=True)

valid_gen = ImageDataGenerator()
train_generator = gen.flow(x_train, y_train, batch_size=batch_size)
validation_generator = valid_gen.flow(x_test, y_test, batch_size=batch_size)

In [0]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', 
                              factor=0.1, 
                              patience=3, 
                              mode='auto', 
                              min_delta=0.0001, 
                              cooldown=0, 
                              min_lr=0)

In [0]:
#checkpoint = GoogleDriveCheckpoint('Ass3C_weights.h5', drive, save_best_only=True, save_weights_only=True, monitor='val_loss',
#                                       verbose=1)

#logger = CSVLogger(os.path.join("./training", "training_24x24.log"))

In [0]:
checkpoint = ModelCheckpoint('gdrive/My Drive/Weights/weights_ASS3C.{epoch:04d}-{val_loss:.2f}.hdf5',
                             monitor='val_loss', 
                             save_best_only=True )

In [0]:
import os
logger = CSVLogger(os.path.join("gdrive/My Drive/Weights/", "training_24x24.log"))

In [0]:
callbacks_list = [reduce_lr,checkpoint,logger]

In [0]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks_list)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 239s 5ms/step - loss: 1.2717 - acc: 0.5546 - val_loss: 2.0683 - val_acc: 0.4780
Epoch 2/50
50000/50000 [==============================] - 223s 4ms/step - loss: 0.7950 - acc: 0.7266 - val_loss: 1.4176 - val_acc: 0.6207
Epoch 3/50
50000/50000 [==============================] - 222s 4ms/step - loss: 0.6196 - acc: 0.7898 - val_loss: 0.9194 - val_acc: 0.7231
Epoch 4/50
50000/50000 [==============================] - 221s 4ms/step - loss: 0.5012 - acc: 0.8293 - val_loss: 0.7888 - val_acc: 0.7517
Epoch 5/50
50000/50000 [==============================] - 221s 4ms/step - loss: 0.4168 - acc: 0.8576 - val_loss: 0.7281 - val_acc: 0.7769
Epoch 6/50
50000/50000 [==============================] - 220s 4ms/step - loss: 0.3492 - acc: 0.8810 - val_loss: 0.5909 - val_acc: 0.8

In [0]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


10000/10000 [==============================] - 17s 2ms/step
Test loss: 0.4339426311135292
Test accuracy: 0.8728
Saved the model to disk


In [0]:
model.save_weights("Yolo_Basic_model2.h5")
print("Saved the model to disk")
from google.colab import files

files.download('Yolo_Basic_model2.h5')

Saved the model to disk
